In [2]:
%cd ..

/home/alta/Conversational/OET/al826/2022/shortcuts/OOD_generalisation


In [3]:
from src.analysis.model_analyser import ModelAnalyser

system_1 = ModelAnalyser('trained_models/baselines/imdb/0')
system_vecs_1 = system_1.get_cls_vectors('imdb', mode='dev', lim=500, quiet=True)

system_2 = ModelAnalyser('trained_models/multi_task/imdb_sst/0')
system_vecs_2 = system_2.get_cls_vectors('imdb', mode='dev', lim=500, quiet=True)

###################
imdb
###################


Reusing dataset imdb (/home/alta/Conversational/OET/al826/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1403.48it/s]


###################
imdb
###################


Reusing dataset imdb (/home/alta/Conversational/OET/al826/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1412.23it/s]


In [ ]:
system_3 = ModelAnalyser('trained_models/baselines/imdb/0')
system_vecs_3 = system_3.get_cls_vectors('imdb', mode='dev', lim=500, quiet=True)


In [5]:
import numpy as np
from src.analysis.CKA import kernel_CKA

def measure_similarity(vecs_1:list, vecs_2:list)->float:
    assert vecs_1.keys() == vecs_2.keys()
    ex_ids = vecs_1.keys()

    W1 = np.array([vecs_1[i] for i in ex_ids])
    W2 = np.array([vecs_2[i] for i in ex_ids])
    return kernel_CKA(W1, W2)

measure_similarity(system_vecs_1, system_vecs_2)

0.9445331523069929

In [4]:
import numpy as np

def measure_similarity(vecs_1:list, vecs_2:list)->float:
    assert vecs_1.keys() == vecs_2.keys()
    ex_ids = vecs_1.keys()
    
    W1 = np.array([vecs_1[i] for i in ex_ids])
    W2 = np.array([vecs_2[i] for i in ex_ids])

    W1_c = W1 - np.mean(W1, axis=0)
    W2_c = W2 - np.mean(W2, axis=0)
    
    W1_sim = np.matmul(W1_c.T, W1_c).reshape(-1)
    W2_sim = np.matmul(W2_c.T, W2_c).reshape(-1)

    W1_norm = W1_sim/np.linalg.norm(W1_sim)
    W2_norm = W2_sim/np.linalg.norm(W2_sim)

    score = np.dot(W1_norm, W2_norm)
    return score

measure_similarity(system_vecs_2, system_vecs_2)

0.9999998

In [5]:
import numpy as np

ex_ids = system_vecs.keys()
vectors = np.array([system_vecs[i] for i in ex_ids])
vecs = np.array(vectors)
print(vecs.shape)

NameError: name 'system_vecs' is not defined

In [ ]:
print(np.mean(vecs, axis=0).shape)

In [ ]:
import math 
import numpy as np

def centering(K):
    n = K.shape[0]
    unit = np.ones([n, n])
    I = np.eye(n)
    H = I - unit / n

    return np.dot(np.dot(H, K), H)  # HKH are the same with KH, KH is the first centering, H(KH) do the second time, results are the sme with one time centering
    # return np.dot(H, K)  # KH


def rbf(X, sigma=None):
    GX = np.dot(X, X.T)
    KX = np.diag(GX) - GX + (np.diag(GX) - GX).T
    if sigma is None:
        mdist = np.median(KX[KX != 0])
        sigma = math.sqrt(mdist)
    KX *= - 0.5 / (sigma * sigma)
    KX = np.exp(KX)
    return KX


def kernel_HSIC(X, Y, sigma):
    return np.sum(centering(rbf(X, sigma)) * centering(rbf(Y, sigma)))


def linear_HSIC(X, Y):
    L_X = np.dot(X, X.T)
    L_Y = np.dot(Y, Y.T)
    return np.sum(centering(L_X) * centering(L_Y))


def linear_CKA(X, Y):
    hsic = linear_HSIC(X, Y)
    var1 = np.sqrt(linear_HSIC(X, X))
    var2 = np.sqrt(linear_HSIC(Y, Y))
    return hsic / (var1 * var2)


def kernel_CKA(X, Y, sigma=None):
    hsic = kernel_HSIC(X, Y, sigma)
    var1 = np.sqrt(kernel_HSIC(X, X, sigma))
    var2 = np.sqrt(kernel_HSIC(Y, Y, sigma))
    return hsic / (var1 * var2)
